In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchsummary import summary
import pickle

# 加载训练数据以验证形状
with open('../dataset/train.pkl', 'rb') as f:
    X_train, y_train = pickle.load(f)

print("原始数据形状:")
print("X_train shape:", X_train.shape)    # 应为 (12800, 96, 96) 而不是之前的 15999
print("y_train shape:", y_train.shape)    # 应为 (12800,)

# 添加通道维度
X_train = X_train[..., np.newaxis]  
X_train = np.transpose(X_train, (0, 3, 1, 2))  

print("\nPyTorch格式数据形状:")
print("X_train shape:", X_train.shape)

In [ ]:
class InvertedResidual(nn.Module):
    def __init__(self, in_c, out_c, stride, expand_ratio):
        super().__init__()
        hidden_dim = in_c * expand_ratio
        self.use_res = stride == 1 and in_c == out_c
        
        layers = []
        if expand_ratio != 1:
            layers.extend([
                nn.Conv2d(in_c, hidden_dim, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True)
            ])
        layers.extend([
            nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True),
            nn.Conv2d(hidden_dim, out_c, 1, bias=False),
            nn.BatchNorm2d(out_c)
        ])
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res:
            return x + self.conv(x)
        return self.conv(x)

In [ ]:
class LightGestureNet(nn.Module):
    def __init__(self, num_classes=8):
        super().__init__()
        
        self.first = nn.Sequential(
            nn.Conv2d(1, 16, 3, 2, 1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU6(inplace=True)
        )
        
        self.layers = nn.Sequential(
            InvertedResidual(16, 24, 2, 6),
            InvertedResidual(24, 24, 1, 6),
            InvertedResidual(24, 32, 2, 6),
            InvertedResidual(32, 32, 1, 6)
        )
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32, num_classes)
        )
        
    def forward(self, x):
        x = self.first(x)
        x = self.layers(x)
        x = self.classifier(x)
        return x

In [ ]:
# 创建模型实例
model = LightGestureNet()

# 打印模型结构
print(model)

# 验证模型输入输出形状
sample_input = torch.randn(1, 1, 96, 96)  # 批次大小为1的样例输入
print("\n验证单个样例输入输出:")
print("输入形状:", sample_input.shape)
output = model(sample_input)
print("输出形状:", output.shape)

# 使用torchsummary显示详细参数信息
print("\n模型详细参数信息:")

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 创建模型并移动到对应设备
model = LightGestureNet().to(device)

# 使用torchsummary显示模型的详细信息
summary(model, input_size=(1, 96, 96), device=str(device))